<a href="https://colab.research.google.com/github/mhtn-it/6_APP_Project/blob/main/Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Đồ án môn Lập trình song song ứng dụng
**Giảng viên: Thầy Trần Trung Kiên**

## Thông tin nhóm
### STT: Nhóm 6

### Thành viên:
1. 1712258 - Nguyễn Văn Hậu - [kenneth-nguyenn](https://github.com/kenneth-nguyenn)
2. 18120181 - Nguyễn Thị Cẩm Hồng - [chnhgr](https://github.com/chnhgr)
3. 18120216 - Mai Huỳnh Trung Nguyên - [mhtn-it](https://github.com/mhtn-it)

### Tài liệu của nhóm
- Kế hoạch nhóm, phân chia công việc: [Google Sheet](https://docs.google.com/spreadsheets/d/1lNRWbRRnsN0L1bEBLm2tHkauJC2jS9DPZiZfhLQ9Av8/edit?usp=sharing)
- Link Colab thực thi: [Google Colab](https://colab.research.google.com/github/mhtn-it/6_APP_Project/blob/main/Report.ipynb)
- Drive làm việc chung: [Google Drive](https://drive.google.com/drive/folders/1bDjdUTDdbr1EV_9VxSVIOGB60OutJlZi?usp=sharing)

## **Mô tả ứng dụng:** 

### **Mô tả tổng quan:**

**Đề tài**: Thay đổi background ảnh dựa vào phương pháp Poisson Matting

**Input**: 
- Một bức ảnh chân dung
- Trimap của ảnh (được tạo từ một bài toán khác)
- Ảnh nền mới mong muốn

**Output**: 
- Ảnh đã được thay đổi nền

**Ý nghĩa thực tế của ứng dụng**:
- Thay đổi background của ảnh khi cần thiết, ví dụ trong các trường hợp như ảnh kỷ niệm, ảnh thẻ, trang trí...
- Áp dụng trên các phần mềm chỉnh sửa ảnh, trang web chỉnh sửa ảnh online

**Lý do cần tăng tốc**: 
- Toàn bộ quá trình xử lý để cho ra một bức ảnh đúng và chính xác có thể mất đến vài phút (giảm trải nghiệm người dùng)
- Khi xử lý hàng loạt, số lượng lớn thì thời gian sẽ rất lâu

### **Ý tưởng chính của bài toán:**

Từ hình ảnh gốc, trimap, ảnh nền mới ứng dụng sử dụng các biện pháp phủ (matting) và pha trộn (blending) để ghép ảnh chân dung background mới.

![demo](https://github.com/avani17101/Poisson-Matting/raw/main/attachments/result1.png)

Qua nhiều phương pháp cải thiện thì foreground ngày càng chính xác với đối tượng của mình.

![demo2](https://github.com/avani17101/Poisson-Matting/raw/main/attachments/result2.png)

### **Phần nhóm dự định song song hoá:**

Nhóm nhận thấy việc xử lý ảnh có thể song song hoá bằng cách chia nhỏ công việc cho từng pixel. Do đó nhóm dự định sẽ lựa chọn các bước:
- Thao tác matting và blending
- Các bước xử lý để tạo độ chính xác cho foreground

## **Giải quyết bài toán**

### **Phần bổ trợ**

Phần này dùng để tạo cách hàm bổ trợ cho việc thực hiện trên Google Colab. Bao gồm: liên kết Google Colab với Drive, Tạo hàm Python để hỗ trợ đọc file pnm và file ma trận ở notebook, kiểm tra thông tin GPU.

#### Liên kết colab với Google Drive
Để tiết kiệm thời gian load dữ liệu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Kiểm tra thông tin GPU

In [ ]:
from numba import jit, prange, cuda

In [ ]:
cuda.detect()

Found 1 CUDA devices
id 0             b'Tesla T4'                              [SUPPORTED]
                      compute capability: 7.5
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True

### **Thực thi tuần tự**

#### Chuẩn bị

Chuyển vị trí thực thi về thư mục Code

In [ ]:
cd drive/MyDrive/6_APP_Project/Code

/content/drive/MyDrive/6_APP_Project/Code


Import thư viện

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import os
from  scipy import ndimage
import time
import scipy
from scipy.optimize import fmin
IMAGES_FOLDER_PATH = "../Data/Input"

#### Quy trình

![Global Matting](https://github.com/mhtn-it/6_APP_Project/blob/main/Data/Report/GlobalMatting.png?raw=true)

#### Các ảnh Input

Danh sách các input, bao gồm:
- Ảnh gốc
- Ảnh trimap
- Ảnh background mới

Lưu ý:
- Ảnh grayscale được thêm vào để minh hoạ, người dùng không cần cung cấp.
- Phương pháp tạo ảnh trimap nhóm đã tìm hiểu và làm thử nhưng chưa thành công nên tạm thời bỏ qua và sử dụng các trimap có sẵn.

<img src='https://drive.google.com/uc?id=1ULspQFfkDyjg-WFZZnLmNRQCjKW1hhhJ'/>

#### Phân tích ảnh input

Từ các ảnh input ta dùng hàm tính toán mask để phân tích ra được vùng/ảnh của foreground, background và unknown. Nguyên tắc: các pixel trên ảnh trimap sẽ là:
- Background: nếu giá trị < 10.
- Foreground: nếu giá trị > 245.
- Unknown: còn lại

<img align="justify" src='https://drive.google.com/uc?id=1S36DiTkgMNj_aUYCNK8es4yvYu5dFmdA'/>


Từ các vùng ở trên ta ghép:
- F = foreground + unknown
- B = background + unknown

<img align="justify" src='https://drive.google.com/uc?id=1_uwZGhYmlF76OxKPetgzvz8ed24asQEJ'/>


#### Global Matting

Phương trình Poisson:

$$\nabla I = (F - B)\nabla \alpha + \alpha \nabla F + (1 - \alpha)\nabla B$$

Suy ra:

$$\Delta \alpha = div (\frac{\nabla I}{F-B})$$
trong đó, $\Delta = (\frac{\partial^2}{\partial x^2},\frac{\partial^2}{\partial y^2})$
là một toán tử Laplacian and toán tử vector div của Gradient ảnh input đối với hiệu của Foreground và Background.


Phương trinh Poisson được giải bằng phương pháp ước lượng Gauss Siedel Iteration

cho,  $$ \frac{\partial^2 f}{\partial x^2}+\frac{\partial^2 f}{\partial y^2} = S$$
$$ {f_{i,j}}^{(n+1)} = \frac{\beta}{4}({f_{i+1,j}}^{(n)} + {f_{i-1,j}}^{(n+1)}) + {f_{i,j+1}}^{(n)} + {f_{i,j-1}}^{(n+1)} - S_{i,j}) + (1-\beta){f_{i,j}}^{(n)}$$
với $\beta$ nằm trong đoạn (1,2)

Tìm hiểu thêm về phương trình Poisson [tại đây](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/poissonmatting_siggraph04.pdf)..


Lý do sử dụng Phương trình Poisson:

Poisson là một phương pháp tổng quát dùng để mô tả hành vi của các đại lượng ví dụ như hướng thế năng điện, trọng lực, chất lỏng. Áp dụng với bài toán này ta có thể mô tả xu hướng gradient của các pixel. Khi đó việc giải phương trình (tìm alpha) có thể xác định được độ mờ của pixel đó để biết nó có xu hướng nghiêng về background hay foreground.

Sau khi áp dụng phương pháp trên ta được một mảng numpy chứa các giá trị alpha dùng để ghép với nền mới. Ta có thể xem alpha nhưng một hệ số làm mờ.

<img src='https://drive.google.com/uc?id=1up3_IVpNZRzD-mziEwlr2M-Kjn_Xrh-4'/>


#### Alpha Blending

Tiến hành blending mảng alpha global matting ở trên với hình nên mới để được kết quả.

<img src='https://drive.google.com/uc?id=1O9bYFhq6t6jfgrDaqglRTurH8NZt8xSm'/>

#### Local Matting

Người dùng sẽ quan sát ảnh sau khi dùng Global Matting và đánh dấu những điểm ROI (Region of Interest - Những khu vực mong muốn được cải thiện hơn) sau đó chương trình sẽ chạy Local Matting trên khu vực này để cải thiện chất lượng hình ảnh.
Local Matting cũng chính là Global Matting nhưng trên một khu vực nhỏ hơn.

Ví dụ, với hình ở trên ta chọn 2 khu vực có chỉ số như sau:
- top, bottom, left, right = [347, 475, 130, 195]
- top, bottom, left, right = [367, 480, 386, 439]


<img src='https://drive.google.com/uc?id=1s3DTgKOwGPqLOCmo_oZV0XncFOdW5Xfz'/>

Local matte và ảnh kết quả sau khi blending:

<img src="https://drive.google.com/uc?id=1St6RIQp6ztso5_k6gOSkqhCEmT_narm_"/>



Hiện tại nhóm đang tự quan sát và set cứng các giá trị ROI trong file code. Nhóm kì vọng cuối đồ án sẽ thực hiện để xử lý qua tham số dòng lệnh để thực tế hơn.

Quy trình dự định:
- In ra ảnh từ global matting
- Người dùng nhập vào danh sách những khu vực muốn chỉnh sửa
- In ra kết quả sau khi local matting

#### Thực hiện

Hiện tại để tiện kiểm tra thì nhóm set cứng giá trị input (ảnh gốc, trimap, background) tương tự như phần trình bày phía trên của notebook. Sau này dự kiến sẽ chuyển qua dùng tham số dòng lệnh.

Thực thi với Python thuần (thời gian xử lý với 1 ảnh):

In [ ]:
%%time
!python3 sequential.py

====SUCCESS GLOBAL MATTING====
====SUCCESS LOCAL MATTING====
CPU times: user 136 ms, sys: 33.7 ms, total: 170 ms
Wall time: 23.5 s


Thực thi tuần tự với Numba jit. Phần sử dụng git:
- Hàm `global_alpha_matting`: hàm chạy phương pháp Gauss Siedel để tìm ra alpha phù hợp.

Những phần sử dụng thư viện có sẵn:
- Numpy: các phương thức `gradient`, `maximum`, `minimun`...; các phép toán nhân, cộng ma trận
- OpenCV: đọc ảnh, lưu ảnh, chuyển ảnh sang grayscale, RGB

> Nhóm đã có thử cài đặt để dùng jit với các hàm trên nhưng thấy tốc độ giảm không đáng kể (< 0.5s hoặc đôi khi chậm hơn) nên quyết định không jit nữa

In [ ]:
%%time
!python3 sequential_jit.py

====SUCCESS GLOBAL MATTING====
====SUCCESS LOCAL MATTING====
CPU times: user 31.4 ms, sys: 5.8 ms, total: 37.2 ms
Wall time: 2.73 s


### Thực thi song song

Phần thực hiện song song: Bước tính toán alpha nằm trong hàm `global_alpha_matting`.

Ý tưởng của hàm `global_alpha_matting`:
  
    Parameters:
    alpha: giá trị xấp xỉ của alpha
    d2alpha: tổng đạo hàm riêng bậc 2 của alpha theo X và Y
    unknown_seg: mảng munby đại diện cho vùng chưa xác định rõ
    iters: số lần chạy của phương pháp ước lượng Gauss Siedel
    threshold: ngưỡng cho sự thay đổi sau mỗi lần chạy. Nếu dưới ngưỡng thì phương trình sẽ dừng lại
    beta: nhân tử beta của phương pháp ước lượng Gauss Siedel

    Return:
    Mảng numpy đại diện cho chỉ số matte đã được tạo

- Bước đầu gán prev_alpha: 1 ma trận toàn số 0
- Tính toán sai khác giữa prev_alpha và alpha
- Đi vào vòng lặp với số lần lặp là iters
  - Tính độ khác biệt giữa prev_alpha và alpha
  - Chạy 2 vòng lặp for lần lượt duyệt qua từng pixel của alpha
  - Tính lại giá trị của từng pixel theo công thức từ beta, alpha, pre_alpha, d2alpha (*)

=> Nhóm lựa chọn song song phần (*) ở trên bằng cách: chia cho mỗi thread đảm nhận các pixel để tính lại giá trị của alpha.

In [ ]:
%%time
!python3 parallel_v1.py

====SUCCESS GLOBAL MATTING====
Traceback (most recent call last):
  File "parallel_v1.py", line 285, in <module>
    local_matte[top:bottom+1, left:right+1] = local_matting(all_data_2, top, bottom, left, right)
  File "parallel_v1.py", line 256, in local_matting
    print(required_alpha.shape())
TypeError: 'tuple' object is not callable
CPU times: user 264 ms, sys: 47.2 ms, total: 312 ms
Wall time: 52.5 s


So sánh độ chính xác:

In [ ]:
img_seq = cv2.imread(os.path.join(IMAGES_FOLDER_PATH, '../Output/output_4_global_jit.png'))
img_seq = cv2.cvtColor(img_seq, cv2.COLOR_BGR2RGB)
img_par = cv2.imread(os.path.join(IMAGES_FOLDER_PATH, '../Output/output_4_global_parallel.png'))
img_par = cv2.cvtColor(img_par, cv2.COLOR_BGR2RGB)
errorL2 = cv2.norm(img_seq, img_par, cv2.NORM_L2)
similarity = 1 - errorL2 / ( img_seq.shape[0] * img_seq.shape[1] )
print('Similarity = ',similarity)

Similarity =  1.0


### Đánh giá

Các kết quả thử nghiệm của nhóm đến thời điểm hiện tại:


**Về mặt độ chính xác:** Ảnh từ các phiên bản cho ra kết quả giống nhau.

**Về mặt thời gian:**

Chỉ dùng Global Matting:
- Python dùng thư viện: 23.6s
- Python thuần dùng jit: 1.53s
- Song song với cuda.jit: 54s

Sử dụng cả Global và Local Matting:
- Python dùng thư viện: 23.5s
- Python thuần dùng jit: 2.73s
- Song song với cuda.jit: Bị lỗi truy xuất vùng nhớ không liên tục


Giải thích các vấn đề gặp phải:
- Phần Global bị tốn thời gian: Do chạy vòng lặp với `iters = 50` nên dữ liệu bị chuyển qua chuyển lại giữa device và host nhiều lần => mất thời gian 

=> Nhóm sẽ thay đổi bằng cách cố gắng thực hiện toàn bộ hàm trên cùng 1 bên, hoặc sử dụng SMEM
- Phần Local chưa thực hiện được: Do quá trình dùng local thì sẽ trích xuất 1 phần nhỏ của toàn bộ ảnh để xử lý. Nhóm dự đoán bị lỗi trong quá trình trích mảng này.

### **Dự định của nhóm sắp tới**

- Tìm hiểu thêm về trimap
- Cải tiến lại phần song song hoá `global matting`
- Song song hoá phần `local matting`
- Thêm các phiên bản cải tiến bằng việc sử dụng SMEM

## Nhìn lại quá trình làm đồ án

**1712898 - Nguyễn Văn Hậu**
- Khó khăn
    + 
    + 
    + 
- Học được:
    + 
    + 

**18120181 - Nguyễn Thị Cẩm Hồng**
- Khó khăn
    + 
    + 
    + 
- Học được:
    + 
    + 

**18120216 - Mai Huỳnh Trung Nguyên**
- Khó khăn
    + 
    + 
    + 
- Học được:
    + 
    + 
    + 

**Nhóm**: 
- Khó khăn:
  + 
  + 
  + 
- Do đó, nếu có thêm thời gian nhóm sẽ cố gắng thực hiện:
  + 
  + 
  + 


## Tài liệu tham khảo

- Project: Poisson Matting, [github](https://github.com/avani17101/Poisson-Matting).
- Paper: Poisson Matting của Juan Sun, Jiaya Jia, Chi-Keung Tang, Heung-Yeung Shum, [paper](http://www.cse.cuhk.edu.hk/~leojia/all_final_papers/matting_siggraph04.pdf)